In [16]:
from NER import NER
from Graph import from_query
import pandas as pd
import spacy
from spacy import displacy
from difflib import get_close_matches
from column import give_column
from isCount import isCount

def remove_(select,where):
    res=[ele if ele not in where else None for ele in select]
    while(None in res):
        res.remove(None)
    return res

Data_Final=pd.read_csv("Augmented_Non_Complex.csv")

Natural=Data_Final['Natural'][1]
# Natural=["Display the authors who on the field Machine Translation and are from affiliate 'The MITRE Corporation'."]
#Natural = ["How many number of conference have been conducted in the field 'F-7'?"]
#Natural = ["What is ID of author 'Mungi, Ashish'"]


SQL=Data_Final['SQL'][1]
# SQL=["SELECT DISTINCT COUNT(*) FROM AuthID_FieldID JOIN AuthID_Name ON AuthID_FieldID.AuthID=AuthID_Name.AuthID WHERE AuthID_FieldID.FieldID='Lexical Acquisition'"]
#SQL = ["SELECT DISTINCT COUNT(*) FROM ConfID_FieldID WHERE ConfID_FieldID.FieldID='F-7';"]


Natural_after_NER=list(map(lambda x: NER(x)[1],[Natural]))
Where_Clause=list(map(lambda x: NER(x)[0],[Natural]))
# Where_ColName=list(map(lambda x: NER(x)[0][1],Natural))

### Predicting SQL Queries

In [17]:
Select_Clause=list(map(give_column,Natural_after_NER))
Is_Count=list(map(isCount,Natural_after_NER))
Select_Clause_Processed=[]

print('Natural Language Statement:')
print(Natural)
print("########################################################################")

print('Actual SQL Query:')
print(SQL)
print("########################################################################")

print('Predicted SQL query:')

for i in range(len(Select_Clause)):
    if(Is_Count[i]):
        if(Where_Clause[i]==[]):
            Select_Clause_Processed.append(Select_Clause[i])
            query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Select_Clause_Processed[i][0])
            print("SELECT DISTINCT COUNT(*) ",query,sep='')
        else:
            Select_Clause_Processed.append(remove_(Select_Clause[i],Where_Clause[i][0][0]))
    #         print(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
            query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
            print("SELECT DISTINCT COUNT(*) ",query," WHERE ",last_table+'.'+Where_Clause[i][0][0],"=","'",Where_Clause[i][0][1],"'",sep='')
    else:
        if(Where_Clause[i]==[]):
            Select_Clause_Processed.append(Select_Clause[i])
            query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Select_Clause_Processed[i][0])
            print("SELECT DISTINCT ",first_table,".",Select_Clause_Processed[i][0]," ",query,sep='')
        else:
            Select_Clause_Processed.append(remove_(Select_Clause[i],Where_Clause[i][0][0]))
#             print(Select_Clause_Processed[i])
            if(len(Select_Clause_Processed[i])==0):
                print("Fuck")
            else:
    #         print(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
                query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
                print("SELECT DISTINCT ",first_table,".",Select_Clause_Processed[i][0]," ",query," WHERE ",last_table+'.'+Where_Clause[i][0][0],"=","'",Where_Clause[i][0][1],"'",sep='')

Natural Language Statement:
List all the authors IDs
########################################################################
Actual SQL Query:
SELECT DISTINCT PaperID_AuthID.AuthID FROM PaperID_AuthID;
########################################################################
Predicted SQL query:
SELECT DISTINCT AuthID_AffiliationID.AuthID FROM AuthID_AffiliationID


### Actual SQL Queries

In [ ]:
list(SQL)

["SELECT DISTINCT COUNT(*) FROM ConfID_FieldID WHERE ConfID_FieldID.FieldID='F-7';"]

### Natural Language Statement

In [ ]:
Natural_after_NER

['How many number of conference have been conducted in the field  ']

### Task: Which columns to select 

In [ ]:
Select_Clause

[['FieldID', 'Name', 'ConfID']]

### Task: Predicting the where clause

In [ ]:
Where_Clause

[[['FieldID', 'F-7']]]

In [ ]:
Select_Clause_Processed

[['Name', 'ConfID']]

In [ ]:
list(Natural)

["How many number of conference have been conducted in the field 'F-7'?"]

In [ ]:
give_column("Which conference was held at and in")

['ConfID']

### Prediction Pipeline

In [18]:
def remove_(select,where):
    res=[ele if ele not in where else None for ele in select]
    while(None in res):
        res.remove(None)
    return res

In [19]:
def NLP_TO_SQL_Pipeline(natural, sql):

  print('Natural Language Statement:')
  print(natural)
  print("########################################################################")

  print('Actual SQL Query:')
  print(sql)
  print("########################################################################")

  print('Predicted SQL query:')

  Natural_after_NER=list(map(lambda x: NER(x)[1],natural))
  Where_Clause=list(map(lambda x: NER(x)[0],natural))

  Select_Clause=list(map(give_column,Natural_after_NER))
  Is_Count=list(map(isCount,Natural_after_NER))
  Select_Clause_Processed=[]

  for i in range(len(Select_Clause)):
    if(Is_Count[i]):
        if(Where_Clause[i]==[]):
            Select_Clause_Processed.append(Select_Clause[i])
            query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Select_Clause_Processed[i][0])
            print("SELECT DISTINCT COUNT(*) ",query,sep='')
        else:
            Select_Clause_Processed.append(remove_(Select_Clause[i],Where_Clause[i][0][0]))
    #         print(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
            query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
            print("SELECT DISTINCT COUNT(*) ",query," WHERE ",last_table+'.'+Where_Clause[i][0][0],"=","'",Where_Clause[i][0][1],"'",sep='')
    else:
        if(Where_Clause[i]==[]):
            Select_Clause_Processed.append(Select_Clause[i])
            query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Select_Clause_Processed[i][0])
            print("SELECT DISTINCT ",first_table,".",Select_Clause_Processed[i][0]," ",query,sep='')
        else:
            Select_Clause_Processed.append(remove_(Select_Clause[i],Where_Clause[i][0][0]))
#             print(Select_Clause_Processed[i])
            if(len(Select_Clause_Processed[i])==0):
                print("Fuck")
            else:
    #         print(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
                query,first_table,last_table=from_query(Select_Clause_Processed[i][0],Where_Clause[i][0][0])
                print("SELECT DISTINCT ",first_table,".",Select_Clause_Processed[i][0]," ",query," WHERE ",last_table+'.'+Where_Clause[i][0][0],"=","'",Where_Clause[i][0][1],"'",sep='')

In [20]:
nlp = ["How many total papers are published?"]
sql = ["SELECT DISTINCT COUNT(*) FROM KeywordID_PaperID;"]

NLP_TO_SQL_Pipeline(nlp, sql)

Natural Language Statement:
['How many total papers are published?']
########################################################################
Actual SQL Query:
['SELECT DISTINCT COUNT(*) FROM KeywordID_PaperID;']
########################################################################
Predicted SQL query:
SELECT DISTINCT COUNT(*) FROM ConfID_PaperID
